# Assignment 10

### 1. Import the data 

In [1]:
import pandas as pd
#from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet

import os
import folium as fm
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap, StripePattern

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString


import branca as br 

In [2]:
#os.chdir("C:\\Users\\User\\Documents\\GitHub\\Diplomado_PUCP\\Lecture_11\\Assigment_10")

In [3]:
# Reconocemos el encode y recién una vez detectado se abre la base
base = open(r'../../_data/institutos1.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']
charenc

'ascii'

In [4]:
institutos = pd.read_csv( r'../../_data/institutos1.csv', encoding = charenc)
institutos

,cod_mod,ltimoden_metaatencion,cuentadeid_postulante_procesoadm,sumaden_flagingresante,sumaden_flagmatriculado,ratio,ratio1,dif,dif1,nlat_ie,...,population_ccpp,altitude,Dblock1km,Dblock2km,Dblock3km,Dblock5km,Dblock10km,Dblock20km,Dblock30km,_merge
0,207597,307,481,302,301,0.980456,1.566775,-6,174,-13.52107,...,106400.0,3386.0,14840369,742184,494123,296073,148036,7418,4912,matched (3)
1,207613,240,502,204,204,0.850000,2.091667,-36,262,-13.63997,...,45864.0,2342.0,14970470,748235,499156,299094,149047,7423,4915,matched (3)
2,239970,537,1047,484,476,0.886406,1.949721,-61,510,-15.47827,...,216716.0,3831.0,17000163,850081,566054,340032,170016,8508,5605,matched (3)
3,262311,423,526,353,349,0.825059,1.243499,-74,103,-6.23041,...,23202.0,2358.0,6781025,339512,226341,135205,67102,3351,2234,matched (3)
4,273979,425,544,411,405,0.952941,1.280000,-20,119,-6.49004,...,67362.0,309.0,7070857,353428,235285,141171,70085,3542,2328,matched (3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1468594,1566,250,241,202,0.128991,0.159642,-1364,-1316,-12.02987,...,579561.0,125.0,13190935,659467,439311,263187,131093,6546,4331,matched (3)
211,1468610,284,74,71,71,0.250000,0.260563,-213,-210,-12.08215,...,66171.0,115.0,13250934,662467,441311,265186,132093,6646,4431,matched (3)
212,1545623,210,266,184,177,0.842857,1.266667,-33,56,-16.13940,...,1175.0,3780.0,17740208,887104,591069,354041,177020,8810,5906,matched (3)
213,1595495,206,144,141,137,0.665049,0.699029,-69,-62,-18.01370,...,86095.0,577.0,19810177,990088,660059,396035,198017,9908,6605,matched (3)


In [5]:
inst_geo = gpd.GeoDataFrame(institutos, crs = "EPSG:4326", geometry= gpd.points_from_xy(institutos.nlong_ie, institutos.nlat_ie))

C:\Users\dell\.conda\envs\myenv\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


In [6]:
shp_dpt = gpd.read_file(r'../../_data/INEI_LIMITE_DEPARTAMENTAL/INEI_LIMITE_DEPARTAMENTAL.shp')

In [7]:
intersct_inst_geo = gpd.overlay(inst_geo, shp_dpt, how = "intersection")

In [8]:
# variables: meta de admisión, total postulantes, total ingresantes, total matriculados
var_sum = ["ltimoden_metaatencion", "cuentadeid_postulante_procesoadm", "sumaden_flagingresante", "sumaden_flagmatriculado"] 

In [9]:
tot_dpt = intersct_inst_geo.groupby(["CCDD"], as_index= False)[var_sum].sum()
#Se hace merge para volver archivo a geopandas
tot_dpt_shp = shp_dpt.merge(tot_dpt, on = "CCDD")

In [10]:
list(tot_dpt_shp.columns)

['OBJECTID_1',
 'OBJECTID',
 'CCDD',
 'NOMBDEP',
 'CAPITAL',
 'Shape_STAr',
 'Shape_STLe',
 'ORIG_FID',
 'Shape_Leng',
 'Shape_Area',
 'CORREO',
 'CONTACTO',
 'WHATSAPP',
 'geometry',
 'ltimoden_metaatencion',
 'cuentadeid_postulante_procesoadm',
 'sumaden_flagingresante',
 'sumaden_flagmatriculado']

In [11]:
#Colorear segun vacantes de institutos

### 2. Import the data

## Parte 2.

Make an interactive map of markes using visual_html function and show total vacancies, applicants, entrants, and enrolled information in a table popup. Additionally, use "university" as icon.
Take this map as an example

In [12]:
# We set the coordinates and the map
inst_geo[ 'lon' ] = inst_geo.geometry.x
inst_geo[ 'lat' ] = inst_geo.geometry.y

zoom_start        = 12
lat_institutes    = inst_geo[ "lat" ].mean()
long_institutes   = inst_geo[ "lon" ].mean()


map2 = fm.Map( location      = [ lat_institutes, long_institutes ], 
               tiles         = "OpenStreetMap", 
               zoom_start    = zoom_start, 
               control_scale = True )

# We set the markers
for index, row in inst_geo.iterrows():
    
    # We create the html tables
    html = """<h4>{cod_mod}</h4>
            <table>
                <tr><td>Total Vacancies:</td><td>{ltimoden_metaatencion}</td></tr>
                <tr><td>Total Applicants:</td><td>{cuentadeid_postulante_procesoadm}</td></tr>
                <tr><td>Total Entrants:</td><td>{sumaden_flagingresante}</td></tr>
                <tr><td>Total Enrolled:</td><td>{sumaden_flagmatriculado}</td></tr>
            </table>""".format( **row.to_dict() )
    
    fm.Marker( [ row[ 'lat' ], row[ 'lon' ] ],
               radius     = 50,
               icon       = fm.Icon( color = "black", icon = "university", prefix = 'fa' ),
               color      = "#3186cc",  
               fill       = True,
               fill_color = "#3186cc",
               popup      = fm.Popup( html, max_width = 300 ) )\
      .add_to( map2 )
    
map2

## Parte 3.

Generate the ratio of applicants by vacancies and store it in a new columns called ratio_applicants_vacancies ( total applicants / total vacancies ). Then, generate a cluster map with two groups of marker clusters. The first group should be compound by those institutes where there were more applicants than vacancies. Name this group as Applicant Surplus. The second group should be compound by those institutes where there were more vacancies than applicants. Name this group as Applicant Deficit.

In [13]:
# We create the column "ratio_applicants_vacancies"

inst_geo[ 'ratio_applicants_vacancies' ] = inst_geo[ 'cuentadeid_postulante_procesoadm' ] / inst_geo[ 'ltimoden_metaatencion' ]

# We set the map
map3 = fm.Map( location      = [ lat_institutes, long_institutes ], 
               tiles         = "OpenStreetMap", 
               zoom_start    = zoom_start, 
               control_scale = True )

# We set the markers
applicant_surplus = MarkerCluster( name = 'Applicant Surplus' ).add_to( map3 )
applicant_deficit = MarkerCluster( name = 'Applicant Deficit' ).add_to( map3 )

# We create the clusters
for index, row in inst_geo.iterrows():
    if row[ 'ratio_applicants_vacancies' ] < 1:
        fm.Marker( location = [ row[ 'lat' ], row[ 'lon' ] ],
                      popup = row[ 'ratio_applicants_vacancies' ] )\
          .add_to( applicant_surplus )

for index, row in inst_geo.iterrows():
    if row[ 'ratio_applicants_vacancies' ] >= 1:
        fm.Marker( location = [ row[ 'lat' ], row[ 'lon' ] ],
                   popup    = row[ 'ratio_applicants_vacancies' ] )\
          .add_to( applicant_deficit )

# We add a control legend
fm.LayerControl( collapsed = False )\
  .add_to( map3 )
        
map3